In [39]:
include("../src/HDC.jl")
include("../src/math.jl")
include("../src/experimental.jl")
using DataFrames
using CSV
using JLD
using MySQL
using Distributions

# Create real valued embeddings
Start values between -1 and 1

## esm bow

In [40]:
aa_embeddings = DataFrame(CSV.File("../data/amino_acid_embeddings.csv"))
amino_acids_esm = [only(i) for i in aa_embeddings.protein_ID]
aa_emb = Matrix(aa_embeddings[:, 2:end])
# Create HDVs
HDV_mat_bit = nested_arrays2mat([hdv() for i in 1:size(aa_emb)[2]], true)
# Extend embeddings into hyperdimensional space
AA_bit_esm = permutedims(mat_scaler(aa_emb * HDV_mat_bit, -1, 1, 2))

AA_dict = Dict(zip(amino_acids_esm, [AA_bit_esm[:, i] for i in 1:21]))

Dict{Char, Vector{Float64}} with 21 entries:
  'P' => [-0.272507, 0.950455, 0.20641, 0.483569, 0.716822, -0.253062, 0.340019…
  'K' => [-0.357308, 0.047448, -0.658337, 0.311858, 0.358922, -1.0, 0.129834, 0…
  'M' => [0.812483, -0.0859642, -0.108308, -0.11587, -0.327918, 0.310134, -0.02…
  'F' => [-1.0, -0.396097, 1.0, -0.532459, 0.897286, 0.948265, 1.0, -0.0201386,…
  'I' => [0.584924, 0.051929, 0.0845688, 0.417464, 0.300493, 0.137673, 0.27791,…
  'H' => [1.0, 0.451451, 0.279083, -0.692371, 0.264824, -0.621991, 0.186978, -0…
  'E' => [0.385735, 0.0601354, -0.0297331, -0.0229051, 0.201929, 1.0, -0.747033…
  'W' => [0.854542, -0.243544, -0.897689, -0.200075, 0.265161, 0.235478, -0.314…
  'S' => [0.559583, 0.908844, -1.0, 0.740205, 0.204389, 0.709706, -0.98632, -0.…
  'T' => [0.198339, 0.186578, -0.682256, 0.0652621, 0.210939, 0.888251, -0.4633…
  'C' => [0.701152, -1.0, -0.557424, -1.0, 0.151787, -0.109081, 0.934042, 0.658…
  'X' => [0.312685, 0.243418, 0.206159, -0.0661614, 0.953024, 0.

In [41]:
trimer_hdvs = Dict(aa1 * aa2 * aa3 => 
AA_dict[aa1] .* circshift(AA_dict[aa2], 1) .* circshift(AA_dict[aa3], 2)
for aa1 in amino_acids_esm for aa2 in amino_acids_esm  for aa3 in amino_acids_esm)

Dict{String, Vector{Float64}} with 9261 entries:
  "HTY" => [-0.0779406, -0.0287617, -0.00887482, 0.111371, 0.0137703, 0.123453,…
  "DRR" => [-0.117537, -0.0230643, -0.0177502, 0.00854369, -0.00210406, 0.00155…
  "QAM" => [0.30966, 0.00406682, -0.279273, -0.0154775, 0.0670787, -0.003303, -…
  "WNG" => [0.250638, 0.00385776, 0.0544533, -0.0385286, 0.0280065, -0.0871316,…
  "PPV" => [0.0132416, -0.174302, 0.108492, 0.0893698, 0.16239, 0.0679863, 0.08…
  "WMA" => [-0.0901473, -0.197876, 0.0617268, 0.0216697, -0.0166212, 0.0508113,…
  "TKL" => [0.076832, 0.0267949, 0.0103475, 0.00377044, -0.0505869, 0.318812, 0…
  "MSW" => [-0.000141897, 7.45551e-7, -0.0841169, -0.0282195, 0.217893, -0.0126…
  "XTY" => [-0.0243709, -0.0155081, -0.00655585, 0.0106423, 0.0495554, -0.07108…
  "ETI" => [-0.00719311, 0.00159071, -0.0032449, 0.000811501, 0.00111447, 0.088…
  "KLF" => [-0.0701141, 0.00278033, -0.0577738, 0.0949911, 0.358922, 0.45219, -…
  "AGI" => [-0.0257842, -0.0347241, -0.0894071, -0.0163734, 

In [42]:
function bow_embedder(sequence)
    l = [trimer_hdvs[sequence[i:i+2]] for i in 1:length(sequence)-2]
    v = reduce(.+, l)
    return v
end

bow_embedder (generic function with 1 method)

In [43]:
non_ml = JLD.load("../data/phalp_non_ml.jld")["up_evd_type"]
conn = DBInterface.connect(MySQL.Connection, "127.0.0.1", "root", "DB_Pw123", db = "PhaLP")
data = DataFrame(DBInterface.execute(conn,"SELECT * FROM PhaLP.UniParc;"))
seq = Dict(zip(data[:, 1],data[:,2]))

Dict{String, Vector{UInt8}} with 11549 entries:
  "UPI000178DD30" => [0x4d, 0x41, 0x4b, 0x56, 0x51, 0x46, 0x54, 0x4b, 0x52, 0x5…
  "UPI0013EFDC93" => [0x4d, 0x47, 0x59, 0x4d, 0x59, 0x50, 0x56, 0x50, 0x4b, 0x4…
  "UPI000172D062" => [0x4d, 0x56, 0x49, 0x4d, 0x53, 0x45, 0x46, 0x56, 0x57, 0x4…
  "UPI001463ABBB" => [0x4d, 0x4e, 0x4c, 0x53, 0x41, 0x4e, 0x46, 0x53, 0x4c, 0x4…
  "UPI000232F56D" => [0x4d, 0x41, 0x4c, 0x47, 0x41, 0x50, 0x4d, 0x45, 0x4e, 0x4…
  "UPI0011625D30" => [0x4d, 0x45, 0x47, 0x47, 0x44, 0x4b, 0x4c, 0x51, 0x47, 0x4…
  "UPI0009882324" => [0x4d, 0x44, 0x49, 0x44, 0x52, 0x4e, 0x52, 0x4c, 0x52, 0x5…
  "UPI000CA1D611" => [0x4d, 0x4b, 0x51, 0x41, 0x41, 0x59, 0x47, 0x53, 0x4c, 0x5…
  "UPI0006BC2F8A" => [0x4d, 0x53, 0x46, 0x4b, 0x4d, 0x4b, 0x59, 0x50, 0x49, 0x4…
  "UPI000BBF7878" => [0x4d, 0x54, 0x45, 0x52, 0x56, 0x4c, 0x52, 0x59, 0x44, 0x4…
  "UPI00138B2696" => [0x4d, 0x51, 0x4c, 0x53, 0x52, 0x4b, 0x47, 0x4c, 0x44, 0x4…
  "UPI00025D6AED" => [0x4d, 0x53, 0x4e, 0x52, 0x4e, 0x49, 0x5

In [44]:
embedded_bow_esm = Dict()
non_ml_seq = collect(keys(non_ml))

for i in non_ml_seq
    new_seq = bow_embedder(String(seq[i]))
    embedded_bow_esm[i] = new_seq
end
@save "../data/phalp_bow_esm_real.jld" embedded_bow_esm

## ESM convolutional

In [45]:
data = DataFrame(DBInterface.execute(conn,"SELECT * FROM PhaLP.UniParc;"))
seq = Dict(zip(data[:, 1],data[:,2]))

function convolved_embedding(sequence, tokens, k=3)
    # layer 1
    kmer_hdvs = []
    for i in 1:length(sequence)-k+1
        kmer = sequence[i:i+k-1]
        aa_hdvs = [circshift(tokens[aa], k-l) for (l, aa) in enumerate(kmer)]
        push!(kmer_hdvs, reduce(.*, aa_hdvs))
    end
    
    # layer 2
    conv_kmer_hdvs = []
    for i in 1:length(kmer_hdvs)-k+1
        convolved_kmers = kmer_hdvs[i:i+k-1]
        conv_hdvs = [circshift(convolved_kmers[l], k-l) for (l, km) in enumerate(convolved_kmers)]
        push!(conv_kmer_hdvs, reduce(.*, conv_hdvs))
    end
    
    return reduce(.+, conv_kmer_hdvs)
end

embedded_CNN_esm = Dict()
non_ml_seq = collect(keys(non_ml))
for i in non_ml_seq
    sequence = String(seq[i])
    new_seq = convolved_embedding(sequence, AA_dict)
    embedded_CNN_esm[i] = new_seq
end

In [46]:
@save "../data/phalp_cnn_esm_real.jld" embedded_CNN_esm

## Random real bow

In [35]:
data = DataFrame(DBInterface.execute(conn,"SELECT * FROM PhaLP.UniParc;"))
seq = Dict(zip(data[:, 1],data[:,2]))
AA_list = ['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y', 'X']

AA_dict = Dict(zip(AA_list, [rand(Uniform(-1,1),10000) for i in AA_list]))

Dict{Char, Vector{Float64}} with 21 entries:
  'M' => [0.438716, 0.152832, 0.0110771, 0.00677782, 0.971969, -0.135213, -0.48…
  'K' => [-0.488423, -0.450633, -0.334681, -0.708983, 0.544928, 0.537998, -0.15…
  'P' => [-0.7464, 0.330163, -0.467727, 0.448434, -0.581428, 0.461746, -0.84186…
  'Q' => [-0.660554, -0.35179, 0.132468, -0.197248, 0.661514, 0.994665, -0.6012…
  'I' => [0.291717, 0.976697, 0.102317, -0.314818, 0.821215, -0.151135, 0.16365…
  'H' => [0.664631, 0.999789, 0.442368, 0.215911, -0.395218, -0.513158, -0.5227…
  'E' => [-0.194628, -0.69415, 0.878643, -0.358664, 0.355544, -0.189454, -0.581…
  'W' => [0.562192, 0.0114708, -0.937221, -0.881339, -0.572418, -0.877474, 0.91…
  'S' => [-0.0828748, -0.425852, -0.685893, 0.437952, 0.958066, 0.712524, 0.331…
  'T' => [0.9324, -0.0392689, -0.0112462, 0.906185, -0.776391, -0.0548371, -0.8…
  'C' => [-0.49666, -0.208356, -0.193512, 0.471485, 0.906146, -0.951737, 0.7862…
  'X' => [0.333813, 0.168534, -0.189265, 0.866808, -0.380929, 0.

In [36]:
trimer_hdvs = Dict(aa1 * aa2 * aa3 => 
AA_dict[aa1] .* circshift(AA_dict[aa2], 1) .* circshift(AA_dict[aa3], 2)
for aa1 in AA_list for aa2 in AA_list  for aa3 in AA_list)


function bow_embedder(sequence)
    l = [trimer_hdvs[sequence[i:i+2]] for i in 1:length(sequence)-2]
    v = reduce(.+, l)
    return v
end

bow_embedder (generic function with 1 method)

In [37]:
embedded_bow_rand = Dict()
non_ml_seq = collect(keys(non_ml))

for i in non_ml_seq
    new_seq = bow_embedder(String(seq[i]))
    embedded_bow_rand[i] = new_seq
end
@save "../data/phalp_bow_rand_real.jld" embedded_bow_rand

## Random real cnn


In [38]:
data = DataFrame(DBInterface.execute(conn,"SELECT * FROM PhaLP.UniParc;"))
seq = Dict(zip(data[:, 1],data[:,2]))

embedded_CNN_rand = Dict()
non_ml_seq = collect(keys(non_ml))
for i in non_ml_seq
    sequence = String(seq[i])
    new_seq = convolved_embedding(sequence, AA_dict)
    embedded_CNN_rand[i] = new_seq
end

@save "../data/phalp_cnn_rand_real.jld" embedded_CNN_rand